# CodeMind fine tuning
## Methods:
* model: bert-base-uncased
* PEFT, QLoRA

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/CodeMind

/content/drive/MyDrive/CodeMind


In [3]:
!pip install -r requirements.txt --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 

In [4]:
import datasets
import evaluate
import numpy as np
import torch
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorWithPadding, \
    BitsAndBytesConfig, DataCollatorForLanguageModeling
from google.colab import userdata

In [6]:
import wandb

wandb.login()

wandb.init(
    project='BERT-base',
    name = '4 bit qlora',
)

wandb: Currently logged in as: jehwan-kim (codemind). Use `wandb login --relogin` to force relogin


In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = 'bert-base-uncased'
token = userdata.get('HF_READ')

tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    quantization_config=bnb_config,
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,839,732 || trainable%: 0.2684930076122181


In [9]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'

df = datasets.load_dataset('kreimben/leetcode_with_youtube_captions', split='train')
df = df.remove_columns(['id', 'thumbnail', 'title_x', 'title_y', 'java', 'c++', 'python', 'javascript', 'similar_question_ids', 'success_rate', 'total_submission', 'total_accepted', 'question_likes', 'question_dislikes',])

def formatting_func(d):
    return f"<human>: {d['question_content']}\nTag: {d['tag']}\nLevel: {d['level']}\nHints: {d['question_hints']}\n<assistant>: {d['cc_content']}".strip()

def tokenize(example):
    full_prompt = formatting_func(example)
    tokenized = tokenizer(full_prompt, padding='max_length', truncation=True)
    return tokenized

df = df.map(tokenize)
df = df.remove_columns(['cc_content', 'question_content', 'tag', 'level', 'question_hints'])
df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18136 [00:00<?, ? examples/s]

Map:   0%|          | 0/18136 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18136
})

In [10]:
ta = TrainingArguments(
    output_dir='out',
    overwrite_output_dir=True,

    fp16=True,
    optim="paged_adamw_8bit",
    lr_scheduler_type='cosine',
    warmup_ratio=.05,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,

    logging_steps=1,
    report_to='wandb',

    # max_steps=1000,
)

In [11]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=df,
    args=ta,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
trainer.train()

Step,Training Loss
1,15.791400
2,16.193100
3,16.586900
4,15.808600
5,16.134000
6,16.104400
7,15.867800
8,16.489800
9,15.705800
10,16.346700


TrainOutput(global_step=13602, training_loss=0.7130540424413949, metrics={'train_runtime': 20753.7735, 'train_samples_per_second': 2.622, 'train_steps_per_second': 0.655, 'total_flos': 1.4374840941379584e+16, 'train_loss': 0.7130540424413949, 'epoch': 3.0})

In [14]:
# upload the trained model to huggingface.
peft_model_id = 'kreimben/CodeMind'
write_token = userdata.get('HF_WRITE')
revision_id = 'bert-base-uncased-20240405'

In [15]:
model.push_to_hub(
    repo_id=peft_model_id,
    token=write_token,
    revision=revision_id,
)

README.md:   0%|          | 0.00/192 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kreimben/CodeMind/commit/758410bb66f5cde3ffd0041723b997d5b346619a', commit_message='Upload model', commit_description='', oid='758410bb66f5cde3ffd0041723b997d5b346619a', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
# del trainer
# del data_collator
# del ta
# del df
# del model
# del tokenizer
# del model_id
# del bnb_config
# del lora_config

# import gc
# import torch

# gc.collect()
# torch.cuda.empty_cache()